# importazione librerie

In [11]:
import pandas as pd
#import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt 
import folium 
import math
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances,haversine_distances
from typing import List
from collections import defaultdict 
from tqdm.notebook import tqdm
import networkx as nx
from math import radians
import json
from math import pi

# importazione dataset e caratterizzazione dati

In [12]:
#Read the dataset 

fields = ["city","lat","lng","country","population"]
df = pd.read_excel('worldcities.xlsx', usecols=fields,engine="openpyxl")
airports_df= pd.read_csv('airports.csv')
Paths_df= pd.read_csv('routes.csv')
number_city = len(df)
#sostituisco il nome della prima colonna del data frame airport_df per comodità (tolgo lo spazio)
aux_list = list(airports_df.columns)
aux_list[0] = 'Airport_ID'
airports_df.columns = aux_list
#elimino gli spazi dai nomi delle colonne di Paths_df
Paths_df.columns =[column.replace(" ", "_") for column in Paths_df.columns]
#caratterizzo i dati 
#elimino da paths_df tutte le colonne che hanno dati rilevanti mancanti
Paths_df = Paths_df[(Paths_df['Destination_airport_ID'] != '\\N') & (Paths_df['Source_airport_ID'] != '\\N')]
#converto i dati nelle colonne "Source airport ID", "Destination airport ID" in interi
Paths_df[["Source_airport_ID", "Destination_airport_ID"]] = Paths_df[["Source_airport_ID", "Destination_airport_ID"]].apply(pd.to_numeric)
#creo le colonne 'Latitude_rad' 'Longitude_rad' applicando la funzione radians(x) alle colonne "Latitude" "Longitude"
#questo per evitare in futuro di doverle calcolare più volte inutilmente
airports_df['Latitude_rad'] = airports_df["Latitude"].apply(lambda x: radians(x))
airports_df['Longitude_rad'] = airports_df["Longitude"].apply(lambda x: radians(x))
df['lat_rad'] = df['lat'].apply(lambda x: radians(x))
df['lng_rad'] = df['lng'].apply(lambda x: radians(x))

# creazioni funzioni richiamabili ausiliarie

In [13]:
def int_to_list (a):
    #converte un intero in una lista con quell'intero come unico elemento
    if type (a) == int:
        return([a])
    else:
        return(a)
def List_comparison (a,b):
    #prende in input due liste, se hanno almeno un elemento in comune restituisce True, altrimenti False
    result = False
    for e in a:
        if e in b:
            result = True
    return(result)

# passo 1 associazione per ogni città degli aereoporti corrispondenti o limitrofi

In [14]:
#passo 1.1b versione ottimizzata
#l'ottimizzazione consisite nel creare un dizionario ausiliario contenente come chiavi i criteri di ricerca che 
#mi servono in airports_df (città e paese) e come valori gli aereoporti associato

lista = [[]] *(len(df.index))
dictionary_aux ={}
for e in tqdm(airports_df.index):
    try:
        dictionary_aux[(airports_df.loc[e,'City'],airports_df.loc[e,'Country'])].append(airports_df.loc[e,'Airport_ID'])
    except KeyError:
        dictionary_aux[(airports_df.loc[e,'City'],airports_df.loc[e,'Country'])] = [airports_df.loc[e,'Airport_ID']]
        
for e in tqdm(df.index):
        try:
            lista[e] = dictionary_aux[(df.loc[e,'city'],df.loc[e,'country'])]
        except KeyError:
            lista[e] = []
df['airport_id'] = lista
#l'aumento di performance è evidente...

In [15]:
#passo 1.2b versione ottimizzata
#per tutte le città senza aeroporto calcolo l'areoporto più vicino nello stesso paese
#se non esiste un aeroporto in quel paese cerco l'aeroporto più vicino
#questa volta creo prima un dizionario dove la chiave è uno stato e i valori sono gli aereoporti di quello stato
df['own_airport'] = np.ones(len(df.index)) 
aux_list = list(set(airports_df['Country']))
aux_dictionary_1 = {}
for e in tqdm(aux_list):
        aux_dictionary_1[e] = list(airports_df[airports_df['Country'] == e].index)
    
    
for e in tqdm(df.index):
    if df.loc[e,'airport_id'] ==[]:
        country = df.loc[e].country
        #aux_airports contiene tutti gli aeroporti presenti nel paese della città che sto analizzando
        try:
            aux_airports = airports_df.loc[aux_dictionary_1[country]]
            aux_airports['distance'] = [(haversine_distances([[df.loc[e,'lat_rad'],df.loc[e,'lng_rad']],
                                                         [aux_airports.loc[x,'Latitude_rad'],
                                                          aux_airports.loc[x,'Longitude_rad']]])[0][1])
                                    for x in aux_airports.index]
            
            df.loc[e,'airport_id'] = int(aux_airports[aux_airports.distance ==  aux_airports['distance'].min()].Airport_ID)
            if df.loc[e,'city'] != aux_airports[aux_airports.distance ==  aux_airports['distance'].min()].City.item():
                df.loc[e,'own_airport'] = 0
            #il KeyError si manifesta qualora le rappresentazioni di country non sono coerenti tra i dataframe df e airports_df
            # oppure non ci sono aereoporti in quella nazione.
            #in questo caso inizio a esplorare un area attorno alla latitudine e longitudine che desidero e allargo sempre di
            #più quell'area fino a quando non trovo un aereoporto
        except KeyError:            
            #per ogni aeroporto calcola la distanza da quella città e prelevo solo gli aereoporti di quella 
            #città oppure il più vicino
            lat = df.loc[e,'lat_rad']
            lng = df.loc[e,'lng_rad']
            airports_near_found = False
            width = 0.01
            while airports_near_found == False:
                latmin = lat - width
                latmax = lat + width
                lngmin = lng - width
                lngmax = lng + width
                if lngmin < -pi:
                    lngmin = 2 *pi - lngmin
                    aux_airports = airports_df[(airports_df['Latitude_rad'] > latmin) & (airports_df['Latitude_rad'] < latmax) 
                                                   & ((airports_df['Longitude_rad'] > lngmin) 
                                                   | (airports_df['Longitude_rad'] < lngmax))]
                elif lngmax > pi:
                    lngmax = lngmax - 2 * pi
                    aux_airports = airports_df[(airports_df['Latitude_rad'] > latmin) & (airports_df['Latitude_rad'] < latmax) 
                                                   & ((airports_df['Longitude_rad'] > lngmin) 
                                                   | (airports_df['Longitude_rad'] < lngmax))]
                else:
                    aux_airports = airports_df[(airports_df['Latitude_rad'] > latmin) & (airports_df['Latitude_rad'] < latmax)
                                                   & (airports_df['Longitude_rad'] > lngmin) 
                                                   & (airports_df['Longitude_rad'] < lngmax)]
                if len(aux_airports) == 0:
                    width += 0.01
                else:
                    airports_near_found = True
                    
            aux_airports['distance'] = [haversine_distances([[lat,lng],
                                                         [aux_airports.loc[x,'Latitude_rad'],
                                                          aux_airports.loc[x,'Longitude_rad']]])[0][1]
                                    for x in aux_airports.index]
            
            #ordino gli aeroporti per distanza
            aux_airports = aux_airports.sort_values(by=['distance'], ascending = 0, inplace = False)
            #trovo il primo aeroporto che non appartiene a quella città
            count = 0
            find = 0
            #l'aeroporto più vicino è effettivamente associato alla città analizzata se ne esiste uno oppure 
            #potrebbe essere associato ad un altra città limitrofa
            while find == 0:
                try: 
                    if aux_airports.iloc[count]['City'] != df.loc[e,'city']:
                        find = 1
                except IndexError:
                    find = 1
                if find == 0:
                    count += 1
            #se count >0 ho trovato aereoporti effettivamente associati alla città
            #se count è zero ho trovato l'aereoporto più vicino che tuttavia è in una città limitrofa e devo considerare
            #che devo fare un extra tragitto epr raggiungere l'aereoporto
            if count > 0:
                aux_airports = aux_airports.iloc[:count]
                df.loc[e,'airport_id'] = [a for a in aux_airports['Airport_ID']]
            if count == 0:
                df.loc[e,'own_airport'] = 0
                df.loc[e,'airport_id'] = aux_airports.iloc[0]['Airport_ID']

df['airport_id'] = df['airport_id'].apply(lambda x: int_to_list(x))             

<ipython-input-15-e042fbde47db>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux_airports['distance'] = [haversine_distances([[lat,lng],


In [16]:
#salvo il dataframe ricalcolato per evitare ulteriori inizializzazioni
df.to_csv (r'export_dataframe_1.csv', index = False, header=True)

In [17]:
#riprendo il dataframe già calcolato
df = pd.read_csv('export_dataframe_1.csv')
df['airport_id'] = df['airport_id'].apply(lambda x: json.loads(x))
df['airport_id'] = df['airport_id'].apply(lambda x: int_to_list(x))

# passo 2 associazione dei tragitti per ogni città e creazione del grafo

In [18]:
#passo 2.1b versione ottimizzata
#calcolo la distanza di ogni tragitto
dictionary_airports = {}
Paths_df['path_lenght'] = np.zeros(len(Paths_df.index))
for e in tqdm(airports_df.index):
    dictionary_airports[airports_df.loc[e,'Airport_ID']] = [airports_df.loc[e,'Latitude'],airports_df.loc[e,'Longitude']]
Airports_Available_List = list(dictionary_airports.keys())

#nel caso non ci sia alcun aeroporto correlato alla tratta,sia di partenzia che di arrivo, elimino quella tratta
for e in tqdm(Paths_df.index):
    if Paths_df.loc[e,'Source_airport_ID'] not in Airports_Available_List or Paths_df.loc[e,'Destination_airport_ID'] not in Airports_Available_List:
        Paths_df.drop(e,inplace = True)
    
for e in tqdm(Paths_df.index):
    start = Paths_df.loc[e,'Source_airport_ID']
    end = Paths_df.loc[e,'Destination_airport_ID']
    Paths_df.loc[e,'path_lenght'] = haversine_distances([[dictionary_airports[start][0]
                                                         ,dictionary_airports[start][1]]
                                                        ,[dictionary_airports[end][0]
                                                         ,dictionary_airports[end][1]]])[0][1]

    

In [19]:
#passo 2.2b versione ottimizzata
#ora Path_Df non ha più senso di essere utilizzato, posso creare un dizionario di percorsi. la coppia di chiavi sarà 
#l'aereoporto di partenza mentre il valore sarà il tempo di volo.
earth_diameter =12742 #[km]
Plane_Speed  = 900 #[km/h]
Paths_Dictionary = {}
for e in tqdm(Paths_df.index):
    Paths_Dictionary[Paths_df.loc[e,'Source_airport_ID']] = {}
for e in tqdm(Paths_df.index):
        distance = Paths_df.loc[e,'path_lenght']
        time = distance * (earth_diameter/2) / Plane_Speed #[h]
        Paths_Dictionary[Paths_df.loc[e,'Source_airport_ID']][Paths_df.loc[e,'Destination_airport_ID']] = time
#crea un dizionario dove per ogni aereoporto collego tutte le città raggiungibili
#chiave = airport_id , city   valore = own_airport
Airports_City_Dictionary = {}
for e in tqdm(airports_df.index):
    Airports_City_Dictionary[airports_df.loc[e,'Airport_ID']] = {}
for e in tqdm(df.index):
    airports_list = df.loc[e,'airport_id']
    for h in airports_list:
        Airports_City_Dictionary[h][e] = df.loc[e,'own_airport']
Paths_Available_Dict = {}
for e in tqdm(df.index):
    airports_list = df.loc[e,'airport_id']
    Paths_Available = []
    Paths_Available_Dict[e] = {}
    for value,index in enumerate(airports_list):
        try:
            Paths_Available.append(Paths_Dictionary[airports_list[value]])
        except:
            pass
        
    for f in Paths_Available:
        for pos,key in enumerate(f):
            Paths_Available_Dict[e][key]=f[key] + 2 * df.loc[e,'own_airport']
dictionary = {}
for city in tqdm(df.index):
    dictionary[city] =[]
    for airport_destination in Paths_Available_Dict[city]:    
        for destination_city in Airports_City_Dictionary[airport_destination]:
            dictionary[city].append((destination_city,Paths_Available_Dict[city][airport_destination]+2*df.loc[destination_city,'own_airport']))
    
    

In [20]:
#salvo il dizionario per evitare ulteriori inizializzazioni
with open("dictionary.json", "w") as outfile:
    json.dump(dictionary, outfile)

In [21]:
#riprendo il dizionario già calcolato
with open("dictionary.json", "r") as outfile:
    data = json.load(outfile)
dictionary = {}
for e in data.keys():
    dictionary[int(e)] = data[e]
    

In [22]:
# creation of the graph with Networkx library from the dictionary C 
G = nx.DiGraph()
for city in dictionary:
    G.add_node(city)

for city in dictionary:
    for near_city in dictionary[city]:
        G.add_edge(city,near_city[0],weight = near_city[1])

In [23]:
nx.has_path(G,536,873)

True

In [24]:
my_path=nx.shortest_path(G,536,873, weight="weight")
#ore = nx.path_weight(G,my_path, "weight")

In [25]:
print(my_path)
#for e in my_path:
    #print(dictionary[e],'\n')
df.loc[my_path]

[536, 7631, 873]


,city,lat,lng,country,population,lat_rad,lng_rad,airport_id,own_airport
536,Milan,45.4669,9.1900,Italy,1351562.0,0.793547,0.160396,[1529],1.0
7631,Steinbach am Taunus,50.1678,8.5719,Germany,10682.0,0.875593,0.149608,[340],0.0
873,Olinda,-8.0000,-34.8500,Brazil,921840.0,-0.139626,-0.608247,[2610],0.0
